# Notebook 1: Extracción de Datos

Este notebook realiza la extracción de datos históricos de cotizaciones para las empresas definidas en el archivo de configuración. Se utilizará la biblioteca yfinance para descargar los datos y se guardarán en formato CSV para su posterior análisis.

## Objetivos
- Cargar configuración desde `config.yaml`
- Descargar datos históricos de las empresas seleccionadas
- Guardar los datos en `data/raw/` en formato CSV
- Documentar metadatos de la extracción

In [1]:
# Importación de bibliotecas necesarias
import sys
import os
from datetime import datetime
import pandas as pd
import numpy as np
import yaml
import json

# Agregar la carpeta src al path para poder importar los módulos propios
# Corregido: Obtener ruta absoluta al proyecto (un nivel arriba de notebooks)
notebook_path = os.path.abspath(os.path.dirname('__file__'))
project_path = os.path.abspath(os.path.join(notebook_path, '..'))
sys.path.append(project_path)

print(f"Ruta del proyecto: {project_path}")

# Importar funciones del módulo data_loader
from src.data_loader import cargar_configuracion, validar_ticker, descargar_datos_empresa, obtener_tickers_de_config

Ruta del proyecto: /home/danielbor/Documentos/Universidad/Estadistica/Proyecto/proyecto_inversion


## 1. Cargar Configuración

Se carga la configuración desde el archivo `config.yaml` que contiene los parámetros necesarios para la extracción de datos:

In [2]:
# Cargar configuración
# La ruta debe ser relativa a este notebook
config_path = os.path.join(project_path, 'config.yaml')
config = cargar_configuracion(config_path)

# Mostrar información clave de configuración
print("Configuración cargada exitosamente")
print(f"Período de análisis: {config['periodo_analisis']['fecha_inicio']} - {config['periodo_analisis']['fecha_fin']}")
empresas_info = obtener_tickers_de_config(config)
print(f"Total de empresas a procesar: {len(empresas_info)}")

Configuración cargada exitosamente
Período de análisis: 2022-05-09 - 2025-05-09
Total de empresas a procesar: 10


## 2. Extracción de Datos

En esta sección se descargan los datos históricos de todas las empresas definidas en la configuración:

In [3]:
# Configurar fechas de inicio y fin
fecha_inicio = config['periodo_analisis']['fecha_inicio']
fecha_fin = config['periodo_analisis']['fecha_fin']

# Diccionarios para almacenar datos y metadatos
datos_empresas = {}
metadatos = {
    "fecha_extraccion": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "periodo_analisis": {
        "inicio": fecha_inicio,
        "fin": fecha_fin
    },
    "empresas_procesadas": [],
    "errores": []
}

# Recorrido por todas las empresas para extraer datos
for ticker_symbol, nombre_empresa, sector in empresas_info:
    print(f"\nProcesando: {nombre_empresa} ({ticker_symbol}) - Sector: {sector}")
    
    # Descargar datos históricos
    datos = descargar_datos_empresa(ticker_symbol, fecha_inicio, fecha_fin)
    
    empresa_info = {
        "ticker": ticker_symbol,
        "nombre": nombre_empresa,
        "sector": sector
    }
    
    if datos is not None and not datos.empty:
        # Almacenar datos en el diccionario
        datos_empresas[ticker_symbol] = datos
        
        # Añadir información a metadatos
        empresa_info["registros"] = len(datos)
        empresa_info["fecha_primera_observacion"] = datos.index.min().strftime("%Y-%m-%d")
        empresa_info["fecha_ultima_observacion"] = datos.index.max().strftime("%Y-%m-%d")
        empresa_info["columnas"] = list(datos.columns)
        empresa_info["estado"] = "éxito"
        
        # Mostrar resumen de datos
        print(f"Datos descargados exitosamente para {nombre_empresa}:")
        print(f"  - Periodo: {empresa_info['fecha_primera_observacion']} a {empresa_info['fecha_ultima_observacion']}")
        print(f"  - Total registros: {empresa_info['registros']}")
        print("  - Primeras filas:")
        display(datos.head())
        
    else:
        # Registrar error
        empresa_info["estado"] = "error"
        empresa_info["mensaje_error"] = "No se pudieron descargar datos o no hay datos disponibles"
        metadatos["errores"].append({
            "ticker": ticker_symbol,
            "nombre": nombre_empresa,
            "mensaje": empresa_info["mensaje_error"]
        })
        print(f"ERROR: No se pudieron descargar datos para {nombre_empresa} ({ticker_symbol})")
    
    # Añadir a la lista de empresas procesadas
    metadatos["empresas_procesadas"].append(empresa_info)

# Resumen final
total_exitoso = sum(1 for e in metadatos["empresas_procesadas"] if e["estado"] == "éxito")
total_error = len(metadatos["errores"])

print(f"\nResumen de la extracción:")
print(f"  - Total empresas procesadas: {len(metadatos['empresas_procesadas'])}")
print(f"  - Extracciones exitosas: {total_exitoso}")
print(f"  - Extracciones fallidas: {total_error}")


Procesando: Microsoft (MSFT) - Sector: tecnologia
YF.download() has changed argument auto_adjust default to True
YF.download() has changed argument auto_adjust default to True
Datos descargados para MSFT desde 2022-05-09 hasta 2025-05-09
Datos descargados exitosamente para Microsoft:
  - Periodo: 2022-05-09 a 2025-05-08
  - Total registros: 753
  - Primeras filas:
Datos descargados para MSFT desde 2022-05-09 hasta 2025-05-09
Datos descargados exitosamente para Microsoft:
  - Periodo: 2022-05-09 a 2025-05-08
  - Total registros: 753
  - Primeras filas:


Price,Close,High,Low,Open,Volume
Ticker,MSFT,MSFT,MSFT,MSFT,MSFT
Date,,,,,
2022-05-09,257.907684,265.491483,256.679480,263.249498,47726000
2022-05-10,262.703583,266.846404,258.385308,264.838356,39336400
2022-05-11,253.979324,264.516711,252.760847,258.979959,48975900
2022-05-12,248.910446,253.326205,243.714859,251.191431,51033800
2022-05-13,254.534927,256.406521,248.910449,250.860012,34925100



Procesando: Alphabet (GOOG) - Sector: tecnologia
Datos descargados para GOOG desde 2022-05-09 hasta 2025-05-09
Datos descargados exitosamente para Alphabet:
  - Periodo: 2022-05-09 a 2025-05-08
  - Total registros: 753
  - Primeras filas:
Datos descargados para GOOG desde 2022-05-09 hasta 2025-05-09
Datos descargados exitosamente para Alphabet:
  - Periodo: 2022-05-09 a 2025-05-08
  - Total registros: 753
  - Primeras filas:


Price,Close,High,Low,Open,Volume
Ticker,GOOG,GOOG,GOOG,GOOG,GOOG
Date,,,,,
2022-05-09,112.550911,115.018122,112.020920,112.769374,34520000
2022-05-10,114.044342,116.140911,112.848802,115.493472,31158000
2022-05-11,113.423767,116.120991,113.114236,113.174452,36502000
2022-05-12,112.627548,114.315051,109.594414,111.410314,41464000
2022-05-13,115.966232,117.528330,113.462588,114.304099,29738000



Procesando: Meta Platforms (META) - Sector: tecnologia
Datos descargados para META desde 2022-05-09 hasta 2025-05-09
Datos descargados exitosamente para Meta Platforms:
  - Periodo: 2022-05-09 a 2025-05-08
  - Total registros: 753
  - Primeras filas:
Datos descargados para META desde 2022-05-09 hasta 2025-05-09
Datos descargados exitosamente para Meta Platforms:
  - Periodo: 2022-05-09 a 2025-05-08
  - Total registros: 753
  - Primeras filas:


...


Procesando: Bristol Myers Squibb (BMY) - Sector: salud_farmaceuticas
Datos descargados para BMY desde 2022-05-09 hasta 2025-05-09
Datos descargados exitosamente para Bristol Myers Squibb:
  - Periodo: 2022-05-09 a 2025-05-08
  - Total registros: 753
  - Primeras filas:
Datos descargados para BMY desde 2022-05-09 hasta 2025-05-09
Datos descargados exitosamente para Bristol Myers Squibb:
  - Periodo: 2022-05-09 a 2025-05-08
  - Total registros: 753
  - Primeras filas:


Price,Close,High,Low,Open,Volume
Ticker,BMY,BMY,BMY,BMY,BMY
Date,,,,,
2022-05-09,67.603882,68.472533,66.354095,67.089789,19130700
2022-05-10,67.621620,68.250946,67.036609,68.091398,15343400
2022-05-11,67.497528,68.543452,67.426616,67.595030,13678100
2022-05-12,67.630486,67.665935,66.354102,67.435482,16224500
2022-05-13,67.187286,67.887517,66.939095,67.754566,13773100



Procesando: Eli Lilly (LLY) - Sector: salud_farmaceuticas
Datos descargados para LLY desde 2022-05-09 hasta 2025-05-09
Datos descargados exitosamente para Eli Lilly:
  - Periodo: 2022-05-09 a 2025-05-08
  - Total registros: 753
  - Primeras filas:
Datos descargados para LLY desde 2022-05-09 hasta 2025-05-09
Datos descargados exitosamente para Eli Lilly:
  - Periodo: 2022-05-09 a 2025-05-08
  - Total registros: 753
  - Primeras filas:


Price,Close,High,Low,Open,Volume
Ticker,LLY,LLY,LLY,LLY,LLY
Date,,,,,
2022-05-09,281.398102,286.943737,278.887948,285.562205,3460500
2022-05-10,277.963684,285.484348,277.282627,284.093078,3140800
2022-05-11,278.926910,286.525426,275.336827,275.988698,2625200
2022-05-12,283.898529,284.083387,275.657881,275.657881,2370000
2022-05-13,284.689209,285.831346,279.691070,285.665409,2337700



Resumen de la extracción:
  - Total empresas procesadas: 10
  - Extracciones exitosas: 10
  - Extracciones fallidas: 0


## 3. Guardado de Datos en Formato CSV

Se guardan los datos descargados en formato CSV en la carpeta `data/raw/`:

In [4]:
# Directorio para guardar los datos crudos
datos_crudos_dir = os.path.join(project_path, config['rutas']['datos_crudos'])

# Asegurarse de que el directorio exista
if not os.path.exists(datos_crudos_dir):
    os.makedirs(datos_crudos_dir)
    print(f"Directorio creado: {datos_crudos_dir}")

# Guardar cada DataFrame como archivo CSV con formato específico
for ticker, datos in datos_empresas.items():
    archivo_csv = os.path.join(datos_crudos_dir, f"{ticker}_datos_historicos.csv")
    
    # Resetear el índice para convertir la fecha en una columna
    datos_con_fecha = datos.reset_index()
    
    # Limpiar cualquier nombre de columna duplicado y asegurarse que sean los correctos
    # Esto eliminará filas con información de ticker que a veces incluye yfinance
    if len(datos_con_fecha) > 0 and isinstance(datos_con_fecha.iloc[0, 0], str) and not pd.to_datetime(datos_con_fecha.iloc[0, 0], errors='coerce'):
        # Eliminar la fila con información del ticker
        datos_con_fecha = datos_con_fecha.iloc[1:].copy()
    
    # Ahora nos aseguramos que las columnas tengan los nombres correctos
    datos_con_fecha.columns = ['Date', 'Close', 'High', 'Low', 'Open', 'Volume']
    
    # Asegurarse de que la fecha esté en el formato correcto
    datos_con_fecha['Date'] = pd.to_datetime(datos_con_fecha['Date'])
    
    # Añadir una columna con el ticker para identificar claramente la empresa
    datos_con_fecha['Ticker'] = ticker
    
    # Reordenar las columnas para tener Date primero, luego Ticker, y después los datos
    datos_con_fecha = datos_con_fecha[['Date', 'Ticker', 'Close', 'High', 'Low', 'Open', 'Volume']]
    
    # Guardar sin incluir el índice numérico
    datos_con_fecha.to_csv(archivo_csv, index=False, date_format='%Y-%m-%d')
    print(f"Datos guardados para {ticker}: {archivo_csv}")

print("\nTodos los datos fueron guardados exitosamente en formato CSV con el encabezado:")
print("Date,Ticker,Close,High,Low,Open,Volume")
print("Donde la columna Ticker identifica claramente a qué empresa corresponden los datos.")

Datos guardados para MSFT: /home/danielbor/Documentos/Universidad/Estadistica/Proyecto/proyecto_inversion/data/raw/MSFT_datos_historicos.csv
Datos guardados para GOOG: /home/danielbor/Documentos/Universidad/Estadistica/Proyecto/proyecto_inversion/data/raw/GOOG_datos_historicos.csv
Datos guardados para META: /home/danielbor/Documentos/Universidad/Estadistica/Proyecto/proyecto_inversion/data/raw/META_datos_historicos.csv
Datos guardados para AMZN: /home/danielbor/Documentos/Universidad/Estadistica/Proyecto/proyecto_inversion/data/raw/AMZN_datos_historicos.csv
Datos guardados para NVDA: /home/danielbor/Documentos/Universidad/Estadistica/Proyecto/proyecto_inversion/data/raw/NVDA_datos_historicos.csv
Datos guardados para JNJ: /home/danielbor/Documentos/Universidad/Estadistica/Proyecto/proyecto_inversion/data/raw/JNJ_datos_historicos.csv
Datos guardados para MRK: /home/danielbor/Documentos/Universidad/Estadistica/Proyecto/proyecto_inversion/data/raw/MRK_datos_historicos.csv
Datos guardados p

## 4. Documentación de Metadatos de la Extracción

Se guardan los metadatos de la extracción en formato JSON para documentar el proceso:

In [5]:
# Generar metadatos adicionales
metadatos["resumen"] = {
    "total_empresas": len(metadatos["empresas_procesadas"]),
    "extracciones_exitosas": total_exitoso,
    "extracciones_fallidas": total_error
}

# Función para convertir objetos datetime en cadenas
def convertir_datetime(obj):
    if isinstance(obj, datetime):
        return obj.strftime("%Y-%m-%d %H:%M:%S")
    raise TypeError(f"Objeto de tipo {type(obj)} no es serializable como JSON")

# Guardar metadatos en un archivo JSON
archivo_metadatos = os.path.join(datos_crudos_dir, "metadatos_extraccion.json")
with open(archivo_metadatos, 'w', encoding='utf-8') as f:
    json.dump(metadatos, f, indent=2, ensure_ascii=False, default=convertir_datetime)

print(f"Metadatos de la extracción guardados en: {archivo_metadatos}")

Metadatos de la extracción guardados en: /home/danielbor/Documentos/Universidad/Estadistica/Proyecto/proyecto_inversion/data/raw/metadatos_extraccion.json


In [6]:
# Mostrar resumen detallado de la extracción
print("\nResumen detallado por empresa:")
resumen_df = pd.DataFrame([
    {"Ticker": e["ticker"], 
     "Nombre": e["nombre"], 
     "Sector": e["sector"], 
     "Estado": e["estado"],
     "Registros": e.get("registros", 0),
     "Primera Obs.": e.get("fecha_primera_observacion", "-"),
     "Última Obs.": e.get("fecha_ultima_observacion", "-")
    } for e in metadatos["empresas_procesadas"]
])
display(resumen_df)


Resumen detallado por empresa:


,Ticker,Nombre,Sector,Estado,Registros,Primera Obs.,Última Obs.
0,MSFT,Microsoft,tecnologia,éxito,753,2022-05-09,2025-05-08
1,GOOG,Alphabet,tecnologia,éxito,753,2022-05-09,2025-05-08
2,META,Meta Platforms,tecnologia,éxito,753,2022-05-09,2025-05-08
3,AMZN,Amazon,tecnologia,éxito,753,2022-05-09,2025-05-08
4,NVDA,NVIDIA,tecnologia,éxito,753,2022-05-09,2025-05-08
5,JNJ,Johnson & Johnson,salud_farmaceuticas,éxito,753,2022-05-09,2025-05-08
6,MRK,Merck & Co.,salud_farmaceuticas,éxito,753,2022-05-09,2025-05-08
7,PFE,Pfizer,salud_farmaceuticas,éxito,753,2022-05-09,2025-05-08
8,BMY,Bristol Myers Squibb,salud_farmaceuticas,éxito,753,2022-05-09,2025-05-08
9,LLY,Eli Lilly,salud_farmaceuticas,éxito,753,2022-05-09,2025-05-08


## Conclusiones

- Se extrajeron datos históricos para todas las empresas definidas en la configuración.
- Los datos se almacenaron en archivos CSV individuales en la carpeta `data/raw/`.
- Se generaron metadatos detallados de la extracción para documentar el proceso.
- El siguiente paso será la limpieza y preprocesamiento de estos datos en el notebook 2.